In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [17]:
## Load ANN Trained Model, sacler pickle, one hot pickle
model = load_model('model.h5')

## Load encoders and scaler
with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender = pickle.load(file)

with open("onehot_encoder_geo.pkl", "rb") as file:
    onehot_encoder_geo = pickle.load(file)

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)


input_data = {
    'CreditScore' : 800,
    'Geography' : "France",
    'Gender' : "Male",
    'Age': 40,
    'Tenure' : 3,
    'Balance' : 70000,
    'NumOfProducts': 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary':30000
}



In [18]:
geo_encoded = onehot_encoder_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

c:\Users\SampathKumar\GenAI\vir_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,Male,40,3,70000,2,1,1,30000


In [21]:
""" Encode categorical Vriables"""
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,1,40,3,70000,2,1,1,30000


In [22]:
## concatination with one hot encoded data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,40,3,70000,2,1,1,30000,1.0,0.0,0.0


In [27]:
## Sacling the input data convert it to 0 to 1
input_scaled = scaler.transform(input_df)
input_scaled

array([[ 1.53956105,  0.91324755,  0.10479359, -0.69539349, -0.09770129,
         0.80843615,  0.64920267,  0.97481699, -1.22456561,  1.00150113,
        -0.57946723, -0.57638802]])

In [29]:
## predict if persom leaves the bank or not
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


array([[0.02944664]], dtype=float32)

In [31]:
prediction_probability = prediction[0][0]
prediction_probability

0.029446635

In [ ]:
if prediction_probability > 0.5:
    print("Customer is Stupid")
else:
    print("Customer is Not Stupid")